In [5]:
import numpy as np
import pandas as pd
import gc
import nltk
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import StoppingCriteria, StoppingCriteriaList

from gpt_j.dataset import load_spider_datasets
from gpt_j.gpt_j import GPTJForCausalLM
from t5.inference import evaluate_result
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
_, _, dev_spider = load_spider_datasets()

In [7]:
# parameters

tokenizer_name = 'EleutherAI/gpt-j-6B'
base_model_name = "hivemind/gpt-j-6B-8bit"
result_path = f'results/predicted_result_gptj_base_4shot.txt'

In [8]:
# model

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B", padding_side="left", truncation_side="left")
tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})
# model = AutoModelForCausalLM.from_pretrained(base_model_name)
model = GPTJForCausalLM.from_pretrained(base_model_name, low_cpu_mem_usage=True)
model = model.to("cuda")

In [9]:
class MyStoppingCriteria(StoppingCriteria):

    def __init__(self, stops = []):
        StoppingCriteria.__init__(self)
        self.stops = [stop.to("cuda") for stop in stops]

    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor):
        for stop in self.stops:
            if stop == input_ids[0][-1]:
                return True
        return False

stop_words_ids = [
    tokenizer(stop_word, return_tensors='pt')['input_ids'].squeeze() for stop_word in ["###", "\n"]]
stopping_criteria = StoppingCriteriaList([MyStoppingCriteria(stops=stop_words_ids)])

In [19]:
step = 100
total_len = len(dev_spider)
for batch in range(100, 1100, 100):
    print(batch)
    sql_outputs = []
    for idx in range(0, 100):
        if (batch + idx) > total_len:
            break
        gc.collect()
        torch.cuda.empty_cache()

        # Tokenize prompt, load to GPU with model
        inputs = tokenizer(dev_spider.iloc[idx]['prompt'],
                           return_tensors="pt")
        inputs = {k: v.to("cuda") for k, v in inputs.items()}

        output_tokens = model.generate(
            **inputs,
            temperature=0.1,
            do_sample=False,
            max_length=1024,
            stopping_criteria=stopping_criteria,
            early_stopping=True
            )
        output_text = tokenizer.batch_decode(output_tokens, skip_special_tokens=True)

        # parse outputs from prompt
        for text in output_text:
            extracted_string = text.split('SQL Output:')[-1]
            extracted_string = extracted_string.strip()
            if extracted_string:
                sql_outputs.append(extracted_string)
            else:
                sql_outputs.append('None')

    with open(result_path, 'a', encoding='utf-8') as f:
        for idx, output in enumerate(sql_outputs):
            db_id = dev_spider.iloc[idx]['db_id']
            f.write(output + '\t' + db_id + '\n')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


100


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

200


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

900


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

1000


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [17]:
sql_outputs

['SELECT COUNT(*) FROM singer',
 'SELECT COUNT(*) FROM singer',
 'None',
 'SELECT name, country, age FROM singer ORDER BY age DESC',
 'None',
 'None',
 'SELECT name, song_release_year FROM singer WHERE age = ( SELECT MIN(age) FROM singer )',
 'SELECT song_name, song_release_year FROM singer WHERE age = (SELECT MIN(age) FROM singer)',
 'SELECT country FROM singer WHERE age > 20']

In [21]:
evaluate_result(result_path)

('                     easy                 medium               '
 'hard                 extra                all                 \n'
 'count                233                  388                  '
 '165                  158                  944                 \n'
 '=====================   EXECUTION ACCURACY     =====================\n'
 'execution            0.009                0.005                '
 '0.006                0.000                0.005               \n'
 '\n'
 '====================== EXACT MATCHING ACCURACY =====================\n'
 'exact match          0.013                0.003                '
 '0.000                0.000                0.004               \n'
 '\n'
 '---------------------PARTIAL MATCHING ACCURACY----------------------\n'
 'select               0.500                0.143                '
 '0.111                0.000                0.162               \n'
 'select(no AGG)       0.500                0.143                '
 '0.111                0